In [39]:
import os 
import numpy as np 
import pandas as pd 
import requests 
import sys
from dotenv import load_dotenv

In [31]:
data_path = "nlbse"

df = pd.read_csv(os.path.join(data_path, "issues_train.csv"))

print(df["repo"].unique())



['facebook/react' 'tensorflow/tensorflow' 'microsoft/vscode'
 'bitcoin/bitcoin' 'opencv/opencv']


In [32]:
react = df[df["repo"] == "facebook/react"]
tf = df[df["repo"] == "tensorflow/tensorflow"]
vscode =  df[df["repo"] == "microsoft/vscode"]
btc =  df[df["repo"] == "bitcoin/bitcoin"]
cv = df[df["repo"] == "opencv/opencv"]


In [36]:
def get_github_token():
    token = os.getenv("GITHUB_TOKEN")
    if not token:
        print("Error: GITHUB_TOKEN environment variable is not set.", file=sys.stderr)
        sys.exit(1)
    return token

In [37]:
def fetch_closed_issues(owner: str, repo: str, token: str):
    """
    Fetch closed issues page-by-page until we collect at least 1 non-PR issue.
    Returns a list (possibly length 1).
    """
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/vnd.github+json",
        "User-Agent": "simple-github-issues-script",
    }

    collected = []
    per_page = 100
    page = 1

    while True:
        print(f"Fetching page {page}...", file=sys.stderr)
        url = f"https://api.github.com/repos/{owner}/{repo}/issues"
        params = {
            "state": "closed",
            "per_page": per_page,
            "page": page,
            "sort": "created",
            "direction": "desc",
        }

        response = requests.get(url, params=params, headers=headers, timeout=30)

        if response.status_code == 403 and response.headers.get("X-RateLimit-Remaining") == "0":
            reset = response.headers.get("X-RateLimit-Reset")
            print(
                f"Rate limit reached. Stopping at page {page}. "
                f"X-RateLimit-Reset={reset}",
                file=sys.stderr,
            )
            break

        if response.status_code != 200:
            print(f"GitHub API error: {response.status_code} {response.text}", file=sys.stderr)
            sys.exit(1)

        issues = response.json()

        if not issues:  # no more pages
            break

        page += 1

        collected.append(issues)

    # store raw JSON
    with open('data2.json', 'w') as f:
        json.dump(collected, f, indent=2)

    return collected


In [40]:
owner = "facebook"
repo = "react"
fetch_closed_issues(owner=owner, repo=repo, token=get_github_token())

Error: GITHUB_TOKEN environment variable is not set.


SystemExit: 1